In [1]:
import cv2 as cv
import time
import numpy as np
import glob
import pymavlink

In [3]:
cam = cv.VideoCapture(0)

cv.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv.imshow("test", frame)

    k = cv.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.jpg".format(img_counter)
        cv.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv.destroyAllWindows()

Escape hit, closing...


In [38]:
#TODO CAMERA CALIBRATION
#needs chessboard img i think there is one online
#returns matrix coeff, distortion coeff


 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)


 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 


images = glob.glob('C:\\Class\\CAPSTONE\\images\\*')
numTrue = 0
 
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(
    gray,
    (7, 7),
    cv.CALIB_CB_ADAPTIVE_THRESH +
    cv.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        numTrue = numTrue+1
 
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,7), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
 
cv.destroyAllWindows()
print(numTrue)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


4


In [35]:
##Borrowed from stack overflow
##https://stackoverflow.com/questions/76802576/how-to-estimate-pose-of-single-marker-in-opencv-python-4-8-0
def my_estimatePoseSingleMarkers(corners, marker_size, mtx, distortion):
    '''
    This will estimate the rvec and tvec for each of the marker corners detected by:
       corners, ids, rejectedImgPoints = detector.detectMarkers(image)
    corners - is an array of detected corners for each detected marker in the image
    marker_size - is the size of the detected markers
    mtx - is the camera matrix
    distortion - is the camera distortion matrix
    RETURN list of rvecs, tvecs, and trash (so that it corresponds to the old estimatePoseSingleMarkers())
    '''
    marker_points = np.array([[-marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, -marker_size / 2, 0],
                              [-marker_size / 2, -marker_size / 2, 0]], dtype=np.float32)
    trash = []
    rvecs = []
    tvecs = []
    for c in corners:
        nada, R, t = cv.solvePnP(marker_points, c, mtx, distortion, False, cv.SOLVEPNP_IPPE_SQUARE)
        rvecs.append(R)
        tvecs.append(t)
        trash.append(nada)
    return rvecs, tvecs, trash

In [ ]:

dict = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_APRILTAG_36H11)
params = cv.aruco.DetectorParameters()
capture = cv.VideoCapture(0)

while True:
    ret, img = capture.read()
    if not ret:
        break
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    detector = cv.aruco.ArucoDetector(dict, params)

    corners, ids, rejected = detector.detectMarkers(gray)

    if ids is not None:
        cv.aruco.drawDetectedMarkers(img, corners, ids)
        rvec, tvec, _ = my_estimatePoseSingleMarkers(corners, 0.1, mtx, dist)
        print(rvec, tvec)
        time.sleep(1e-4)
    
    cv.imshow("April Tag:", img)

    if cv.waitKey(1) == ord('q'):
        break

    
capture.release()
cv.destroyAllWindows()



In [29]:
from pymavlink import mavutil


#Create connection from topside
master = mavutil.mavlink_connection('udpin:0.0.0.0:14550')
# master.close_port()
master.close()


In [65]:

from pymavlink import mavutil
import time

protocol = 'tcp'; address = '192.168.2.2'; port = '6777'; connection = f'{protocol}:{address}:{port}'

# print(connection)

#Create connection from topside
master = mavutil.mavlink_connection(connection)

#verify connection
master.wait_heartbeat()


# try:
#     master.wait_heartbeat()
#     print('t')
#     pass
# except:
#     print('e')
#     master.close()

print("messages:", master.messages)
time.sleep(0.1)


messages: {'MAV': <pymavlink.mavutil.mavfile_state object at 0x000002295D8CB550>, 'HOME': <pymavlink.dialects.v20.ardupilotmega.MAVLink_gps_raw_int_message object at 0x000002295D631D10>}


In [45]:
master.close()

In [5]:
import cv2 as cv
#http://192.168.2.2/vehicle/autopilot
# can find the video stream url here in case it changes
capture = cv.VideoCapture("rtsp://192.168.2.2:8554/video_stream__dev_video2")
img_counter = 0

while(1):
    ret, frame = capture.read()
    cv.imshow('frame', frame)

    k = cv.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.jpg".format(img_counter)
        cv.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

capture.release()

cv.destroyAllWindows()

opencv_frame_0.jpg written!
opencv_frame_1.jpg written!
opencv_frame_2.jpg written!
opencv_frame_3.jpg written!
opencv_frame_4.jpg written!
opencv_frame_5.jpg written!
opencv_frame_6.jpg written!
opencv_frame_7.jpg written!
opencv_frame_8.jpg written!
opencv_frame_9.jpg written!
opencv_frame_10.jpg written!
opencv_frame_11.jpg written!
opencv_frame_12.jpg written!
opencv_frame_13.jpg written!
opencv_frame_14.jpg written!
opencv_frame_15.jpg written!
opencv_frame_16.jpg written!
opencv_frame_17.jpg written!
opencv_frame_18.jpg written!
opencv_frame_19.jpg written!
opencv_frame_20.jpg written!
opencv_frame_21.jpg written!
opencv_frame_22.jpg written!
opencv_frame_23.jpg written!
opencv_frame_24.jpg written!
opencv_frame_25.jpg written!
opencv_frame_26.jpg written!
opencv_frame_27.jpg written!
opencv_frame_28.jpg written!
opencv_frame_29.jpg written!
opencv_frame_30.jpg written!
opencv_frame_31.jpg written!
opencv_frame_32.jpg written!
opencv_frame_33.jpg written!
opencv_frame_34.jpg writ

In [54]:
master.close()

In [61]:
import time

from pymavlink import mavutil

def wait_conn():
    msg = None
    while not msg:
        master.mav.ping_send(int(time.time() * 1e6), 0, 0, 0) #microseconds
        msg = master.recv_match()
        print(msg)
        time.sleep(0.5)

master = mavutil.mavlink_connection('udpout:0.0.0.0:14550')
master.mav.heartbeat_send(0,0,0,0,0)
master.wait_heartbeat()
wait_conn()
    


KeyboardInterrupt: 